In [ ]:
import pandas as pd

In [ ]:
main_table = pd.read_excel(r'C:\Users\XXX\excel_file_name.xlsx')

In [ ]:
import sqlalchemy
engine = sqlalchemy.create_engine(r'sqlite:///C:\Users\XXX\XXX\XXX\pandas_database.db')

metadata_obj = sqlalchemy.MetaData()

table = sqlalchemy.Table(
    'test_table',                                        
    metadata_obj,                                    
    sqlalchemy.Column('index', sqlalchemy.Integer),
    sqlalchemy.Column('first_id', sqlalchemy.String, primary_key=True),                    
    sqlalchemy.Column('second_id', sqlalchemy.String),                
)

metadata_obj.create_all(engine)

In [ ]:
no_of_rows = len(main_table)
counter = 0

for row in range (no_of_rows):
    x_to_be_passed = main_table.iloc[counter,0]
    y_to_be_passed = main_table.iloc[counter,1]
    index_to_be_passed = counter
    with engine.connect() as conn:
        conn.execute(sqlalchemy.insert(table).
                     values
                     (index=index_to_be_passed, 
                      first_id= x_to_be_passed, 
                      second_id = y_to_be_passed)
                    )
        conn.commit()
    if counter % 1000 == 0:
        print (f'Already added {counter} rows')
    else:
        pass
    counter +=1
    

In [ ]:
dataframe_from_db = pd.read_sql_table("test_table", con=engine.connect())
dataframe_from_db.head(20)

In [ ]:
###adding values with session - not yet working

import sqlalchemy.orm

with sqlalchemy.orm.Session(engine) as session:
    no_of_rows = len(main_table)
    counter = 0

    for row in range (no_of_rows):
        x_to_be_passed = main_table.iloc[counter,0]
        y_to_be_passed = main_table.iloc[counter,1]
        index_to_be_passed = counter
        session.add(sqlalchemy.insert(table).
                     values
                     (index=index_to_be_passed, 
                      first_id= x_to_be_passed, 
                      second_id = y_to_be_passed)
                    )
        if counter % 1000 == 0:
            print (f'Already added {counter} rows')
        else:
            pass
        counter +=1
    session.commit()

In [ ]:
###experiments with tuple and lists

tuple_ex = (0,1,2)
tuple_ex2 =({0, 2}, {0, 1})

no_of_rows = len(main_table)
counter = 0

tuple_blank = ()

for row in range (100):
    x_to_be_passed = main_table.iloc[counter,0]
    y_to_be_passed = main_table.iloc[counter,1]
    index_to_be_passed = counter
    row = (x_to_be_passed,y_to_be_passed,index_to_be_passed)
    tuple_blank = tuple_blank + row
    counter +=1

tuple_blank

In [ ]:
###not to use

with engine.connect() as conn:
    conn.execute(sqlalchemy.text("CREATE TABLE test_table (index int, x str, y str)"))  ###first transactional statement
    #conn.execute(
        #sqlalchemy.text("INSERT INTO test_table (x, y) VALUES (:x, :y)"),
        #[{"x": 1, "y": 1}, {"x": 2, "y": 4}, {"x": 6, "y": 8}, {"x": 9, "y": 10}])
    conn.commit()

In [ ]:
###to be checked later on - alternative approach

no_of_rows = len(main_table)
counter = 0

index_to_be_passed =""
x_to_be_passed =""
y_to_be_passed =""

for row in range (no_of_rows):
    x_to_be_passed = main_table.iloc[counter,0]
    y_to_be_passed = main_table.iloc[counter,1]
    index_to_be_passed = counter
    with engine.connect() as conn:
        conn.execute(sqlalchemy.text("INSERT INTO test_table (index, x, y) VALUES (:index, :x, :y)"),[{"index": index_to_be_passed, "x": x_to_be_passed, "y": y_to_be_passed}])
        conn.commit()
    counter +=1
    print (counter)
    